In [71]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from datetime import datetime, date
from psx import stocks, tickers
from IPython.display import display
import pandas as pd
import investpy
import yfinance as yf


In [72]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [73]:
display(tickers)

<bound method DataReader.tickers of <psx.web.DataReader object at 0x137733e50>>

In [79]:
# Assuming tickers is your DataFrame
tickers_column = tickers()
symbols = list(tickers_column["symbol"])
# Print or display the tickers column
# ticker_list = list(tickers_column)
print(symbols)

['AGSILSC', 'AKBLTFC6', 'AKBLTFC7', 'BAFLTFC6', 'BAFLTFC7', 'BAFLTFC8', 'BAHLTFC10', 'BAHLTFC9', 'BIPLSC', 'BOPTFC2', 'HBLTFC2', 'HBLTFC3', 'HUBCSC4', 'HUBPHLSC', 'JSBLTFC2', 'JSBLTFC3', 'KELSC5', 'KELSC6', 'MUGHALSC', 'P02PFL060425', 'P02PFL090225', 'P02PFL210925', 'P03PFL090226', 'P03PFL191026', 'P03PFL210926', 'P03PIB040825', 'P03PIB050824', 'P05GIS091225', 'P05GIS290725', 'P05GIS300425', 'P05PFL060428', 'P05PFL161127', 'P05PFL180625', 'P05PFL191028', 'P05PFL210928', 'P05PIB131027', 'P05PIB151025', 'P05PIB190924', 'P05PIB290427', 'P10PFL220829', 'P10PIB101230', 'P10PIB120728', 'P10PIB190929', 'PESC1', 'PESC2', 'PK03TB080224', 'PK03TB110124', 'PK03TB141223', 'PK03TB161123', 'PK03TB220224', 'PK03TB250124', 'PK03TB281223', 'PK06TB020524', 'PK06TB040424', 'PK06TB070324', 'PK06TB080224', 'PK06TB110124', 'PK06TB141223', 'PK06TB160524', 'PK06TB161123', 'PK06TB180424', 'PK06TB210324', 'PK06TB250124', 'PK06TB281223', 'PK06TB300524', 'PK06TB301123', 'PK12TB020524', 'PK12TB031024', 'PK12TB0404

In [ ]:

from matplotlib.dates import relativedelta


data = stocks(symbols, start=date(2020, 1, 1), end=(datetime.now() + relativedelta(days=2)).date())
# data = stocks("786", start=date(2020, 1, 1), end=(datetime.now() + date(2023, 1, 1)))


In [ ]:
display(data)

In [ ]:
investpy.get_commodities()

In [ ]:
from_date = datetime(year=2022, month=1, day=1).strftime("%d/%m/%Y")
to_date = datetime(year=2023, month=1, day=1).strftime("%d/%m/%Y")

investpy.get_commodity_historical_data("GLD", from_date=from_date, to_date=to_date,country="United States")

In [ ]:
wheat_symbol = "WHEATFUTURE.CBT"
rice_symbol = "RICEFUTURE.CBOT"

# Define the date range
start_date = datetime(2023, 1, 1)
end_date = datetime.now() + relativedelta(days=2)

# Download historical data
commodity_data = yf.download("PKR=X", start=start_date, end=end_date)


# Print the historical data
display(commodity_data)

In [ ]:
display(commodity_data)

In [ ]:


def get_commodity_etfs():
    # Get all tickers available on Yahoo Finance
    all_tickers = yf.Tickers

    # Get a list of all tickers
    all_tickers_list = yf.download("AAPL", group_by='ticker').index.get_level_values(1).unique()

    # Filter out tickers related to commodities
    commodity_tickers = [ticker for ticker in all_tickers_list if 'Commodity' in yf.Ticker(ticker).info.get('category', '').lower()]

    return commodity_tickers

# Get the list of commodity-related tickers
commodity_tickers = get_commodity_etfs()

# Print the tickers
for ticker_symbol in commodity_tickers:
    print(ticker_symbol)

In [ ]:
import pandas_datareader as web

tickers = ["ALI=F"]

current_price = list()

for i in range(len(tickers)):
    company = yf.Ticker(tickers[i])
    print(company.info)
# print(current_price)

In [ ]:
()